In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.pylabtools import figsize

plt.style.use('seaborn-darkgrid')
%matplotlib inline

### Load data function
- drop NAN rows
- drop constant columns
- set index column

In [68]:
def filter_csv(df):
    
    df = df.dropna()
    df = df.loc[:, (df != df.iloc[0]).any()]
    
    return df

def load_csv(fname, index_col_name):
    
    df = pd.read_csv(os.path.join(data_dir, fname))
    
    df = filter_csv(df)
    
    df = df.set_index(index_col_name)
    
    return df

### Load train data

In [76]:
data_dir = './data'

train = load_csv('train.csv', 'card_id')
train.index

Index(['C_ID_92a2005557', 'C_ID_3d0044924f', 'C_ID_d639edf6cd',
       'C_ID_186d6a6901', 'C_ID_cdbd2c0db2', 'C_ID_0894217f2f',
       'C_ID_7e63323c00', 'C_ID_dfa21fc124', 'C_ID_fe0fdac8ea',
       'C_ID_bf62c0b49d',
       ...
       'C_ID_598ecf1dfe', 'C_ID_c2124fa8ce', 'C_ID_64545039d3',
       'C_ID_9072609210', 'C_ID_0443db161f', 'C_ID_963962de2c',
       'C_ID_1314773c0b', 'C_ID_7666735b3d', 'C_ID_73f5a0efd0',
       'C_ID_92c9984c58'],
      dtype='object', name='card_id', length=201917)

### Load transactions for each card id of train data

In [80]:
chunksize = 10000
card_id_purchase = {}

# load from historical_transactions.csv
for chunk in pd.read_csv(os.path.join(data_dir, 'historical_transactions.csv'), index_col=1, chunksize=chunksize):

    for card_id in train.index[:10]:
        
        if card_id not in chunk.index: 
            continue
            
        if card_id in card_id_purchase:
            card_id_purchase[card_id].append(chunk.loc[card_id])
        else:
            card_id_purchase[card_id] = [chunk.loc[card_id]]


# load from new_merchant_transactions.csv
for chunk in pd.read_csv(os.path.join(data_dir, 'new_merchant_transactions.csv'), index_col=1, chunksize=chunksize):

    for card_id in train.index[:10]:
        
        if card_id not in chunk.index: 
            continue
            
        if card_id in card_id_purchase:
            card_id_purchase[card_id].append(chunk.loc[card_id])
        else:
            card_id_purchase[card_id] = [chunk.loc[card_id]]
            
print(len(card_id_purchase))


10


In [81]:
# concat 
for k in card_id_purchase:
    card_id_purchase[k] = pd.concat(card_id_purchase[k])

/anaconda/envs/py36/lib/python3.6/site-packages/pandas/core/indexes/api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)


In [82]:
for k in card_id_purchase:
    print(k)
    break

C_ID_7e63323c00


In [85]:
df = card_id_purchase['C_ID_7e63323c00']
df = filter_csv(df)
df.head()

,authorized_flag,city_id,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
card_id,,,,,,,,,,
C_ID_7e63323c00,Y,160,705,M_ID_f7acf6f9d3,-13,-0.708169,2017-01-27 12:27:08,5.0,21,33
C_ID_7e63323c00,Y,160,367,M_ID_915d8da3ad,-1,-0.628784,2018-01-30 18:34:18,5.0,21,16
C_ID_7e63323c00,Y,268,705,M_ID_c59fb0907b,-5,-0.735097,2017-09-14 17:37:57,5.0,21,33
C_ID_7e63323c00,Y,160,705,M_ID_548f18cdd2,-11,-0.725811,2017-03-20 15:07:04,5.0,21,33
C_ID_7e63323c00,Y,160,705,M_ID_548f18cdd2,-13,-0.688124,2017-01-17 14:03:25,5.0,21,33


In [92]:
df['purchase_date'] = pd.to_datetime(df.purchase_date)
df = df.sort_values(by='purchase_date')
df_ = pd.get_dummies(df, columns=['authorized_flag'])
df_.shape

(264, 11)

### Merge merchat information

In [91]:
merchant = pd.read_csv(os.path.join(data_dir, 'merchants.csv'), index_col=0)
merchant = filter_csv(merchant)
print(merchant.shape)

(322802, 20)


In [93]:
m_infos = []
not_in = []
for m_id in df_['merchant_id']:
    if m_id not in merchant.index:
        not_in.append(m_id)
    else:
        m_infos.append(merchant.loc[m_id].to_frame().T)
m_infos = pd.concat(m_infos)
m_infos.shape
len(not_in)

8

In [51]:
m_infos.head()

,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,avg_purchases_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
M_ID_f7acf6f9d3,36446,705,33,-0.0277259,-0.0574706,N,B,B,1,1.00502,...,0.98,0.994477,6,0.99,1.00377,12,Y,160,21,5
M_ID_915d8da3ad,2594,367,16,0.0912529,-0.0574706,N,C,C,1.04,1.06231,...,1.05,1.06769,6,1.12,1.10133,12,Y,160,21,5
M_ID_c59fb0907b,322,705,33,-0.017811,-0.0475558,N,B,B,0.98,0.935957,...,0.99,0.934577,6,1.01,0.955363,12,N,268,21,5
M_ID_548f18cdd2,32531,705,33,0.170572,-0.0574706,N,A,A,1.01,1.00028,...,1.02,1.00291,6,1.05,1.03372,12,Y,160,21,5
M_ID_548f18cdd2,32531,705,33,0.170572,-0.0574706,N,A,A,1.01,1.00028,...,1.02,1.00291,6,1.05,1.03372,12,Y,160,21,5


In [52]:
not_in

['M_ID_1c8dce7fc2',
 nan,
 'M_ID_1c8dce7fc2',
 'M_ID_00a6ca8a8a',
 'M_ID_a6c4d75942',
 'M_ID_a9d91682ad',
 'M_ID_a9d91682ad',
 'M_ID_a9d91682ad',
 'M_ID_a9d91682ad']